In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import joblib
import optuna  # For hyperparameter tuning

# Tickers
tickers = ["BNB-USD", "BTC-USD", "ETH-USD", "XRP-USD"]

# Prepare data with feature engineering
def prepare_data(df):
    features = ["Open", "High", "Low", "Close", "Volume", "Lag1", "Lag7", "SMA7", "RSI14", "MACD", "Returns", "Volatility"]
    X = df[features].values
    y = df["Target"].values
    return X, y, features

# Objective function for Optuna hyperparameter tuning
def objective(trial, X_train_scaled, y_train, X_test_scaled, y_test):
    params = {
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'poly']),
        'C': trial.suggest_float('C', 1e-1, 1e3, log=True),
        'epsilon': trial.suggest_float('epsilon', 0.01, 0.2, log=True),
        'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True) if trial.params['kernel'] == 'rbf' else None
    }
    
    if params['kernel'] == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
    
    model = SVR(**{k: v for k, v in params.items() if v is not None})
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Train and evaluate SVR model
for ticker in tickers:
    print(f"\nTraining SVR for {ticker}...")
    
    # Load data
    df = pd.read_csv(f"{ticker}_processed.csv", index_col="Date", parse_dates=True)
    X, y, feature_names = prepare_data(df)
    
    # Split data (80% train, 20% test, no shuffle for time series)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Scale features
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    
    # Hyperparameter tuning with Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X_train_scaled, y_train, X_test_scaled, y_test), n_trials=30)
    best_params = study.best_params
    print(f"Best parameters for {ticker}: {best_params}")
    
    # Train final model with best parameters
    model = SVR(**best_params)
    model.fit(X_train_scaled, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    test_dates = df.index[train_size:]
    actual_series = pd.Series(y_test, index=test_dates)
    pred_series = pd.Series(y_pred, index=test_dates)
    actual_direction = (actual_series.shift(-1) > actual_series).iloc[:-1].astype(int)
    predicted_direction = (pred_series.shift(-1) > pred_series).iloc[:-1].astype(int)
    directional_accuracy = (actual_direction == predicted_direction).mean() * 100
    
    print(f"SVR Results for {ticker}:")
    print(f"MSE: {mse:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"Directional Accuracy: {directional_accuracy:.2f}%")
    
''' # Save model and scaler
    joblib.dump(model, f"{ticker}_svr_model.pkl")
    joblib.dump(scaler_X, f"{ticker}_scaler_X_svr.pkl")
    print(f"Model and scaler saved for {ticker}")'''

[I 2025-03-11 01:13:44,141] A new study created in memory with name: no-name-5fc787e2-ab8d-4ec7-8624-aa4250e2db7d
[I 2025-03-11 01:13:44,190] Trial 0 finished with value: 2511.7243087259667 and parameters: {'kernel': 'rbf', 'C': 430.1009732024597, 'epsilon': 0.14973795156104544, 'gamma': 0.06259088820119464}. Best is trial 0 with value: 2511.7243087259667.
[I 2025-03-11 01:13:44,211] Trial 1 finished with value: 798.0027773574323 and parameters: {'kernel': 'rbf', 'C': 166.2947222269289, 'epsilon': 0.13861633940964188, 'gamma': 0.005714534935034397}. Best is trial 1 with value: 798.0027773574323.
[I 2025-03-11 01:13:44,227] Trial 2 finished with value: 311633.9828010709 and parameters: {'kernel': 'poly', 'C': 10.579575637754758, 'epsilon': 0.017339997444596977, 'degree': 4}. Best is trial 1 with value: 798.0027773574323.
[I 2025-03-11 01:13:44,246] Trial 3 finished with value: 67880.35828063317 and parameters: {'kernel': 'poly', 'C': 2.013280096818757, 'epsilon': 0.012097583924473704, '


Training SVR for BNB-USD...


[I 2025-03-11 01:13:44,344] Trial 8 finished with value: 5976.979989104645 and parameters: {'kernel': 'rbf', 'C': 16.592597027064254, 'epsilon': 0.02850230943536678, 'gamma': 0.04492565072443325}. Best is trial 1 with value: 798.0027773574323.
[I 2025-03-11 01:13:44,378] Trial 9 finished with value: 1911.0421820347126 and parameters: {'kernel': 'rbf', 'C': 388.1596511546658, 'epsilon': 0.030724204743607483, 'gamma': 0.046881734316870816}. Best is trial 1 with value: 798.0027773574323.
[I 2025-03-11 01:13:44,413] Trial 10 finished with value: 918.4419135066134 and parameters: {'kernel': 'rbf', 'C': 123.56919202283802, 'epsilon': 0.18607110428694507, 'gamma': 0.00024140880512363674}. Best is trial 1 with value: 798.0027773574323.
[I 2025-03-11 01:13:44,442] Trial 11 finished with value: 896.3813522185061 and parameters: {'kernel': 'rbf', 'C': 107.03443925249567, 'epsilon': 0.19931204446221792, 'gamma': 0.0002836741899079831}. Best is trial 1 with value: 798.0027773574323.
[I 2025-03-11 0

Best parameters for BNB-USD: {'kernel': 'rbf', 'C': 988.8914502599667, 'epsilon': 0.021632465365064465, 'gamma': 0.0006951953516197932}
SVR Results for BNB-USD:
MSE: 364.14
MAPE: 2.15%
Directional Accuracy: 47.86%

Training SVR for BTC-USD...


[I 2025-03-11 01:13:45,211] Trial 10 finished with value: 2474625876.7806206 and parameters: {'kernel': 'rbf', 'C': 34.984997598237555, 'epsilon': 0.010113391129683506, 'gamma': 0.0004971129576715602}. Best is trial 2 with value: 338762689.4866304.
[I 2025-03-11 01:13:45,244] Trial 11 finished with value: 1672666890.597372 and parameters: {'kernel': 'rbf', 'C': 777.8946857428747, 'epsilon': 0.027205131857270425, 'gamma': 0.09909548382839062}. Best is trial 2 with value: 338762689.4866304.
[I 2025-03-11 01:13:45,277] Trial 12 finished with value: 856747321.2732526 and parameters: {'kernel': 'rbf', 'C': 926.1035992997995, 'epsilon': 0.021139261660736493, 'gamma': 0.017297810154355824}. Best is trial 2 with value: 338762689.4866304.
[I 2025-03-11 01:13:45,296] Trial 13 finished with value: 2068456564.556353 and parameters: {'kernel': 'rbf', 'C': 122.61912236749662, 'epsilon': 0.01763130219182164, 'gamma': 0.0018269052315705472}. Best is trial 2 with value: 338762689.4866304.
[I 2025-03-11

...

[I 2025-03-11 01:13:45,911] Trial 8 finished with value: 19102.7279174078 and parameters: {'kernel': 'rbf', 'C': 786.0203072271722, 'epsilon': 0.05598406124140807, 'gamma': 0.0003216014734487902}. Best is trial 3 with value: 18387.327349200237.
[I 2025-03-11 01:13:45,931] Trial 9 finished with value: 14922.662750733112 and parameters: {'kernel': 'rbf', 'C': 527.1638487216945, 'epsilon': 0.1113645698736132, 'gamma': 0.00509271393637487}. Best is trial 9 with value: 14922.662750733112.
[I 2025-03-11 01:13:45,961] Trial 10 finished with value: 458805.13484538137 and parameters: {'kernel': 'poly', 'C': 52.000751511761976, 'epsilon': 0.011404261924168983, 'degree': 2}. Best is trial 9 with value: 14922.662750733112.
[I 2025-03-11 01:13:45,994] Trial 11 finished with value: 18374.684486834947 and parameters: {'kernel': 'rbf', 'C': 74.37716226923274, 'epsilon': 0.02781126742052017, 'gamma': 0.003274026005409435}. Best is trial 9 with value: 14922.662750733112.
[I 2025-03-11 01:13:46,011] Tria

Best parameters for ETH-USD: {'kernel': 'rbf', 'C': 527.1638487216945, 'epsilon': 0.1113645698736132, 'gamma': 0.00509271393637487}
SVR Results for ETH-USD:
MSE: 14922.66
MAPE: 3.01%
Directional Accuracy: 50.00%

Training SVR for XRP-USD...


[I 2025-03-11 01:13:46,921] Trial 2 finished with value: 377529846.7911945 and parameters: {'kernel': 'poly', 'C': 741.9452123615205, 'epsilon': 0.09554011058546949, 'degree': 4}. Best is trial 1 with value: 2.6490658083336007.
[I 2025-03-11 01:13:46,943] Trial 3 finished with value: 233622.5154556873 and parameters: {'kernel': 'poly', 'C': 10.152928913855046, 'epsilon': 0.07335590275782554, 'degree': 3}. Best is trial 1 with value: 2.6490658083336007.
[I 2025-03-11 01:13:46,954] Trial 4 finished with value: 2.389916023560206 and parameters: {'kernel': 'rbf', 'C': 2.6390624103015594, 'epsilon': 0.02245548054708987, 'gamma': 0.003332331995526448}. Best is trial 4 with value: 2.389916023560206.
[I 2025-03-11 01:13:47,377] Trial 5 finished with value: 610661.0286487938 and parameters: {'kernel': 'poly', 'C': 22.27316609475408, 'epsilon': 0.03335443345989499, 'degree': 3}. Best is trial 4 with value: 2.389916023560206.
[I 2025-03-11 01:13:47,377] Trial 6 finished with value: 2.738709592323

Best parameters for XRP-USD: {'kernel': 'rbf', 'C': 56.41313809038124, 'epsilon': 0.041198810508855946, 'gamma': 0.00025416411759374006}
SVR Results for XRP-USD:
MSE: 1.34
MAPE: 37.61%
Directional Accuracy: 53.57%


' # Save model and scaler\n    joblib.dump(model, f"{ticker}_svr_model.pkl")\n    joblib.dump(scaler_X, f"{ticker}_scaler_X_svr.pkl")\n    print(f"Model and scaler saved for {ticker}")'